In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import random
from retrying import retry


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

In [144]:


def get_complex_list(auth_key, user):
    down_url = "https://new.land.naver.com/api/complexes/single-markers/2.0?cortarNo=1120010500&zoom=15&priceType=RETAIL&markerId&markerType&selectedComplexNo&selectedComplexBuildingNo&fakeComplexMarker&realEstateType=APT:OPST:ABYG:JGC&tradeType=A1&tag=::::::::&rentPriceMin=0&rentPriceMax=900000000&priceMin=0&priceMax=50000&areaMin=39.7&areaMax=99.2&oldBuildYears&recentlyBuildYears&minHouseHoldCount&maxHouseHoldCount&showArticle=false&sameAddressGroup=false&minMaintenanceCost&maxMaintenanceCost&directions=&leftLon=126.994413&rightLon=127.0859085&topLat=37.5806951&bottomLat=37.5526995"
    r = requests.get(down_url,data={"sameAddressGroup":"false"},headers={
        "Referer": "https://new.land.naver.com/complexes?ms=37.5624147,127.0347404,15&a=APT:OPST:ABYG:JGC&e=RETAIL&g=50000&h=33.1&i=99.2",
        
        "Accept": "*/*",
        "Accept-Encoding": "gzip",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "authorization": auth_key,
        "Host": "new.land.naver.com",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
    #     "sec-ch-ua": "Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105",
        "sec=ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "Windows",
        "User-Agent": f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    })
    r.encoding = "utf-8-sig"
    temp=json.loads(r.text)

    complex_list=[]
    for i in temp:
        complex_list.append(i['markerId'])
    return complex_list
    
def get_article_list(auth_key, user, complex_id):
    down_url = f"https://new.land.naver.com/api/articles/complex/{complex_id}?realEstateType=APT:OPST:ABYG:JGC&tradeType=A1"
    r = requests.get(down_url,data={"sameAddressGroup":"false"},headers={
        "Referer": f"https://new.land.naver.com/complexes/{complex_id}?ms=37.5638922,127.026578,15&a=APT:OPST:ABYG:JGC&b=A1&e=RETAIL&g=50000&h=33.1&i=99.2",
        
        "Accept": "*/*",
        "Accept-Encoding": "gzip",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "authorization": auth_key,
        "Host": "new.land.naver.com",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
    #     "sec-ch-ua": "Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105",
        "sec=ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "Windows",
        "User-Agent": f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    })
    r.encoding = "utf-8-sig"
    temp=json.loads(r.text)

    article_list = []
    for i in temp['articleList']:
        article_list.append(i['articleNo'])
    return article_list

def get_article(auth_key, user, complex_id, article_id):
    down_url = f'https://new.land.naver.com/api/articles/{article_id}?complexNo={complex_id}'
    r = requests.get(down_url,data={"sameAddressGroup":"false"},headers={
        "Referer": f"https://new.land.naver.com/complexes/{complex_id}?ms=37.5638922,127.0176302,15&a=APT:OPST:ABYG:JGC&b=A1&e=RETAIL&g=50000&h=33.1&i=99.2&articleNo={article_id}",
        
        "Accept": "*/*",
        "Accept-Encoding": "gzip",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "authorization": auth_key,
        "Host": "new.land.naver.com",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
    #     "sec-ch-ua": "Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105",
        "sec=ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "Windows",
        "User-Agent": f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    })
    r.encoding = "utf-8-sig"
    temp=json.loads(r.text)
    
    tax = temp['articleTax']
    price = temp['articlePrice']
    space = temp['articleSpace']
    addition = temp['articleAddition']
    floor = temp['articleFloor']
    year = temp['articleDetail']['aptUseApproveYmd']
    
    try:
        monthly = temp['articleDetail']['monthlyManagementCost']
    except:
        monthly= 0
    
    if price['dealPrice'] <  50000 and round(space['exclusiveSpace'] / 3.30579, 2) >= 9 :
#     if True:
        
        info = pd.DataFrame({
            "건물": addition['realEstateTypeName'],
            "거래": addition['tradeTypeName'],
            "최고층": addition['floorInfo'].split("/")[-1],
            "층": addition['floorInfo'].split("/")[0],
            "방향": addition['direction'],
            "사용승인일": f"{year[:4]}-{year[4:6]}-{year[6:]}",
            "관리비": monthly,

            "거래가(만원)": price['dealPrice'],
            "보증금(만원)": price['allWarrantPrice'],
            "월세(만원)": price['allRentPrice'],
            "평당가(만원)": price['priceBySpace'],

            "계약면적(m2/평)": f"{space['supplySpace']} / {round(space['supplySpace'] / 3.30579, 2)}",
            "전용면적(m2/평)": f"{space['exclusiveSpace']} / {round(space['exclusiveSpace'] / 3.30579, 2)}", 

            "취득세(만원)": tax['totalPrice'] * 0.0001, 

            "총 비용(만원)": price['dealPrice'] + tax['totalPrice'] * 0.0001
        }, index=[addition['articleName']])

        return info
    else:
        return 0
        



In [142]:
complex_list = get_complex_list(auth_key, 100)
print(len(complex_list))

139


In [129]:
# complex_list = [
# #     "120864", # 청계센트럴
# #     "119555", # 창보리버리치
# #     "106942", # 청계천제니스
# #     "8619", # 청구
# #     "108854", #동대문푸르지오
# #     "120387", #트레저힐스
# #     "10825", #마일스디오빌(주상복합)
# #     "115109" #그린빌리지
# ]

In [145]:
article_detail = pd.DataFrame()
for idx, i in tqdm(enumerate(complex_list), desc="complex"):
    article_list = get_article_list(auth_key,idx,i)
    for idx_, j in tqdm(enumerate(article_list), desc='article', leave=False):
        new_article = get_article(auth_key,idx_, i,j)
        try:
            article_detail = pd.concat([article_detail, new_article])
        except:
            pass

article_detail = article_detail.sort_values(by=['거래가(만원)'], ascending=False)
article_detail.to_csv("성동.csv")

article_detail_direc = article_detail.loc[(article_detail['방향'] == '남향')]
article_detail_direc.to_csv("남향.csv")

complex: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]

...

article: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]

article: 0it [00:00, ?it/s]